### Задание 4.1 

In [ ]:
select city, count(airport_name) Qty 
from dst_project.airports 
group by 1 
having count(airport_name)>1 
order by 2 desc 

### Задание 4.2 

In [ ]:
#вопрос1 - 6
select count(distinct status) from dst_project.flights

#вопрос2 - 58
select count(status) from dst_project.flights where status='Departed'

#вопрос3 - 402
select count(seat_no) from dst_project.seats where aircraft_code='773'

#вопрос4 - 74227
select count(*)
from dst_project.flights
where (actual_arrival between '2017-04-01' and '2017-09-01') and (status = 'Arrived')


### Задание 4.3 

In [ ]:
#вопрос1 - 437
select count(*) from dst_project.flights where status='Cancelled'

#вопрос2 - 

SELECT 'Airbus', count(*)
FROM dst_project.aircrafts a
WHERE a.model LIKE 'Airbus%'
union all
SELECT 'Boeing', count(*)
FROM dst_project.aircrafts a
WHERE a.model LIKE 'Boeing%'
union all
SELECT 'Sukhoi', count(*)
FROM dst_project.aircrafts a
WHERE a.model LIKE 'Sukhoi%'

#вопрос3 - Europe/Asia
select timezone, count(airport_code) from dst_project.airports group by timezone order by 2 desc


#вопрос4 - 157571
SELECT f.flight_id,  
(f.actual_arrival - f.scheduled_arrival) delta
FROM dst_project.flights AS f WHERE f.actual_arrival IS NOT NULL
order by 2 desc

### Задание 4.4 

In [ ]:
#вопрос1 - 14.08.2016
select scheduled_departure from dst_project.flights order by 1 asc limit 1

#вопрос2 - 530
SELECT EXTRACT(EPOCH FROM AGE(f.scheduled_arrival, f.scheduled_departure))/60 delta, departure_airport, arrival_airport
FROM dst_project.flights AS f WHERE f.scheduled_arrival IS NOT NULL or f.scheduled_arrival is not null
order by 1 desc limit 1 

#вопрос3 - DME -UUS
SELECT EXTRACT(EPOCH FROM AGE(f.scheduled_arrival, f.scheduled_departure))/60 delta, departure_airport, arrival_airport
FROM dst_project.flights AS f WHERE f.scheduled_arrival IS NOT NULL or f.scheduled_arrival is not null
order by 1 desc limit 1 

#вопрос4 - 128
select round(avg(A.delta)) from
(

SELECT EXTRACT(EPOCH FROM AGE(f.scheduled_arrival, f.scheduled_departure))/60 delta, departure_airport, arrival_airport
FROM dst_project.flights AS f WHERE f.scheduled_arrival IS NOT NULL or f.scheduled_arrival is not null
order by 1 desc


) as A

### Задание 4.5 

In [ ]:
#вопрос1 - Economy 85
select distinct B.fare_conditions, count(*)
from 
(
select * from dst_project.aircrafts ac
join dst_project.seats acs on ac.aircraft_code = acs.aircraft_code where ac.aircraft_code='SU9'
) as B 
group by 1
order by 2 desc limit 1

#вопрос2 - 3400
select * from dst_project.bookings order by 3 asc  limit 1

#вопрос3 - 2A
select bp.seat_no from dst_project.tickets t
    join dst_project.boarding_passes bp on bp.ticket_no = t.ticket_no 
    where t.passenger_id = '4313 788533'



# 5. Предварительный анализ

### Задание 5.1 


In [ ]:
#вопрос1 - 486
select count(*) 
    from dst_project.flights f 
    join dst_project.airports a on a.airport_code = f.arrival_airport 
    where a.city = 'Anapa' and date_part('Year',f.actual_arrival)=2017
    
#вопрос2 - 127
select count(*)  
    from dst_project.flights f join dst_project.airports a on a.airport_code = f.arrival_airport
    where a.city = 'Anapa' 
    and date_part('Year',f.actual_arrival)=2017 
    and date_part('Month',f.actual_arrival) IN (11,12,1,2)

#вопрос3 - 1
select count(*)  
    from dst_project.flights f join dst_project.airports a on a.airport_code = f.arrival_airport 
    where a.city = 'Anapa' and f.status='Cancelled'

#Воспро4 - 453
select count(*) 
    from dst_project.flights f join dst_project.airports a on a.airport_code = f.departure_airport 
    where a.city = 'Anapa' and f.arrival_airport not in 
        (select distinct f.arrival_airport 
             from dst_project.flights f join dst_project.airports a on a.airport_code = f.arrival_airport 
             where a.city = 'Moscow')
        
#вопрос5 -
запрос на кол-во мест в каждой модели
select distinct model, count(*) over (partition by model)qty from dst_project.aircrafts ac join dst_project.seats acs on acs.aircraft_code = ac.aircraft_code order by 2 desc

запрос на вылетающие из анапы рейсы с маркой самолета
select *
    from dst_project.flights f 
    join dst_project.airports a on a.airport_code = f.departure_airport 
    join dst_project.aircrafts ac on ac.aircraft_code = f.aircraft_code
    where a.city = 'Anapa'

# собираем датасет

In [ ]:

WITH table_1 AS
  (SELECT f.flight_id,
          f.flight_no,
          f.departure_airport,
          f.arrival_airport,
          f.actual_departure,
          f.status,
          a.aircraft_code
   FROM dst_project.flights f
   LEFT JOIN dst_project.aircrafts a ON f.aircraft_code = a.aircraft_code), /* В таблице table_1 содержится общая информация о рейсах и самолетах. Даты вылета-прилета, код самолетах
id рейса - уникальное значение для каждого рейса
*/ 
   table_2 AS
  (SELECT tf.flight_id,
          count(CASE
                    WHEN tf.fare_conditions = 'Economy' THEN tf.fare_conditions
                END) AS ticket_economy,
          count(CASE
                    WHEN tf.fare_conditions = 'Comfort' THEN tf.fare_conditions
                END) AS ticket_comfort,
          count(CASE
                    WHEN tf.fare_conditions = 'Business' THEN tf.fare_conditions
                END) AS ticket_bisiness
   FROM dst_project.ticket_flights AS tf
   GROUP BY 1), /* В таблице table_2 - информация о заполняемости мест по каждому классу, для каждого рейса */
   
   table_3 AS
  (SELECT flight_id,
          sum(amount) total_amount
   FROM dst_project.ticket_flights
   GROUP BY 1), /* В таблице table_3 - общая стоимость билетов на рейс */
   
   table_4 AS
  (SELECT aircraft_code,
          count(DISTINCT seat_no) cnt_seats
   FROM dst_project.seats
   GROUP BY 1), /* В таблице table_4 - общее количество мест в самолете */
   
   table_5 AS
  (SELECT f.flight_id,
          (EXTRACT(EPOCH
                   FROM f.actual_arrival) - EXTRACT(EPOCH
                                                    FROM f.actual_departure)) / 60 / 60 AS fly_time,
          EXTRACT(ISODOW
                  FROM f.actual_departure) AS dow
   FROM dst_project.flights AS f
   GROUP BY 1) /* В таблице table_5 - время полета и день недели, когда совершен рейс */

SELECT t1.flight_id,
       t1.flight_no,
       t1.departure_airport,
       t1.arrival_airport,
       t1.actual_departure,
       t1.status,
       t1.aircraft_code,
       t2.ticket_economy,
       t2.ticket_comfort,
       t2.ticket_bisiness,
       t3.total_amount,
       t4.cnt_seats,
       t5.fly_time,
       t5.dow
FROM table_1 t1
LEFT JOIN table_2 t2 ON t1.flight_id = t2.flight_id
LEFT JOIN table_3 t3 ON t1.flight_id = t3.flight_id
LEFT JOIN table_4 t4 ON t1.aircraft_code = t4.aircraft_code
LEFT JOIN table_5 t5 ON t1.flight_id = t5.flight_id
WHERE departure_airport = 'AAQ'
  AND (date_trunc('month', actual_departure) in ('2016-12-01',
                                                    '2017-01-01',
                                                    '2017-02-01'))
  AND status not in ('Cancelled')